# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 9.2.2
- **Date :** 11/11/2023
- **Heure :** 12:38
- **Données :** `ready_WindGustDir_WindGustSpeed_WindGustDirNum.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/omar/ready_WindGustDir_WindGustSpeed_WindGustDirNum.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,RainTomorrow,year,month,day,LocationNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.3,18.5,0.0,3.8,8.9,4.0,17.0,82.0,63.0,1017.9,...,0,2008,12,1,36,1.963495,3.926991,-35.024782,117.883608,5
2008-12-02,14.7,21.0,0.0,5.4,9.1,6.0,17.0,64.0,71.0,1012.8,...,0,2008,12,2,36,0.785398,5.890486,-35.024782,117.883608,5
2008-12-03,16.5,24.0,0.2,4.8,10.0,19.0,24.0,36.0,73.0,999.8,...,1,2008,12,3,36,0.392699,3.534292,-35.024782,117.883608,5
2008-12-04,14.1,20.6,7.8,7.0,7.2,13.0,41.0,69.0,81.0,1006.7,...,0,2008,12,4,36,1.570796,3.534292,-35.024782,117.883608,5
2008-12-05,13.0,17.8,0.0,3.6,4.9,13.0,19.0,73.0,63.0,1015.3,...,0,2008,12,5,36,3.926991,3.926991,-35.024782,117.883608,5


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2996 entries, 2008-12-01 to 2017-06-25
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MinTemp        2996 non-null   float64
 1   MaxTemp        2996 non-null   float64
 2   Rainfall       2996 non-null   float64
 3   Evaporation    2996 non-null   float64
 4   Sunshine       2996 non-null   float64
 5   WindSpeed9am   2996 non-null   float64
 6   WindSpeed3pm   2996 non-null   float64
 7   Humidity9am    2996 non-null   float64
 8   Humidity3pm    2996 non-null   float64
 9   Pressure9am    2996 non-null   float64
 10  Pressure3pm    2996 non-null   float64
 11  Cloud9am       2996 non-null   float64
 12  Cloud3pm       2996 non-null   float64
 13  Temp9am        2996 non-null   float64
 14  Temp3pm        2996 non-null   float64
 15  RainToday      2996 non-null   int64  
 16  RainTomorrow   2996 non-null   int64  
 17  year           2996 non-null   int64  
 18

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[17, 20],
         [10, 17],
         [18, 21],
         [18, 23],
         [ 0,  9]]),
  array([[ 4,  7],
         [ 4, 21],
         [14, 21],
         [15, 21],
         [14, 20]]),
  array([[ 3, 22],
         [19, 22],
         [ 7, 17],
         [ 1, 23],
         [11, 17]]),
  array([[ 6, 12],
         [ 4, 22],
         [11, 21],
         [ 3, 16],
         [10, 22]]),
  array([[13, 17],
         [17, 20],
         [13, 18],
         [ 6, 23],
         [ 6, 13]]),
  array([[ 6, 14],
         [ 0,  5],
         [ 5, 17],
         [14, 17],
         [16, 19]]),
  array([[15, 18],
         [ 5, 16],
         [16, 19],
         [ 8, 15],
         [10, 19]]),
  array([[ 1,  9],
         [ 9, 15],
         [ 3, 23],
         [16, 19],
         [17, 20]]),
  array([[10, 15],
         [21, 24],
         [17, 22],
         [ 6, 11],
         [18, 23]]),
  array([[ 9, 20],
         [ 4, 17],
         [ 8, 12],
         [ 4, 14],
         [ 

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,338,6
1,104,51


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.98      0.86       344
           1       0.89      0.33      0.48       155

    accuracy                           0.78       499
   macro avg       0.83      0.66      0.67       499
weighted avg       0.81      0.78      0.74       499

